In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.optim as optim
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np

# Xarxes convolucionals

L'objectiu d'avui és la creació d'una xarxa convolucional que obtengui **com a mínim igual resultat que la xarxa completament connectada implementada la setmana anterior però amb menys paràmetres**. Per poder realitzar comparacions directes emprarem el mateix conjunt de dades.

Com objectius secundaris tenim:

1. Aprenentatge de noves estratègies per evitar `overfitting`.
2. Us d'un nou optimitzador.
3. Visualització dels resultats dels filtres convolucionals.

Primer de tot, com sempre, les dades:


In [2]:
# Recordau: El label del dataset és l'índex de la llista labels. Cada posició de la llista és un codi ASCII. Podeu emprar la funció chr per fer la transformació

# Definim una seqüència (composició) de transformacions
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),  # mitjana, desviacio tipica (precalculats)
    ]
)

# Descarregam un dataset ja integrat en la llibreria Pytorch:
train = datasets.EMNIST(
    "data", split="digits", train=True, download=True, transform=transform
)  ## Si acabau podeu fer proves amb el split "balanced"
test = datasets.EMNIST("data", split="digits", train=False, transform=transform)

100%|██████████| 561753746/561753746 [01:21<00:00, 6912126.56it/s] 


Extracting data\EMNIST\raw\gzip.zip to data\EMNIST\raw


In [3]:
train_batch_size = 64
test_batch_size = 100

# Transformam les dades en l'estructura necessaria per entrenar una xarxa
train_loader = torch.utils.data.DataLoader(train, train_batch_size)
test_loader = torch.utils.data.DataLoader(test, test_batch_size)

## Definició de la xarxa

### Feina a fer

1. Definir la primera xarxa convolucional. A continuació teniu una llista de les capes que podeu emprar:

-   `Conv2d`: Capa convolucional en 2 dimensions ([enllaç](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)). Com a paràmetres principals trobarem:

    -   in_channels: canals d'entrada.
    -   out_channels : canals de sortida.
    -   kernel_size: mida del filtre.
    -   stride: desplaçament del filtre. Típicament pren per valor 1.
    -   padding: ampliació de la imatge per evitar pèrdua de dimensionalitat.

-   `MaxPool2d`: Capa de max pooling ([enllaç](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d)). Aquesta capa no té paràmetres entrenables. Però si:

    -   kernel_size: Mida del filtre del qual es seleccionarà el màxim.
    -   stride: desplaçament del filtre.

-   `Dropout`: Dropout és un mètode de regularització (evitar `overfitting`) que aproxima l'entrenament d'un gran nombre de xarxes neuronals amb diferents arquitectures en paral·lel. Durant l'entrenament, una part de les sortides de la capa s'ignoren aleatòriament o s'abandonen. Això té l'efecte de fer que la capa sembli i es tracti com una capa amb un nombre diferent de nodes i connectivitat a la capa anterior. En efecte, cada actualització d'una capa durant l'entrenament es realitza amb una vista diferent de la capa configurada. Hem d'especificar quines capes tenen `dropout` de manera individual. Té un únic paràmetre amb valor per defecte $p=0.5$ Els valors típics d'aquest paràmetre varien entre $0.5$ i $0.8$.

-   `Linear`

-   `ReLU`

2. Per posibilitar la visualització de les imatges passades per les capes convolucionals farem que funció `forward` tengui diverses sortides (diferents valors de `return`) un per cada capa convolucional de la xarxa.


In [ ]:
in_channels = 3  # Número de canales de entrada (RGB en este caso)
out_channels = 96  # Número de filtros
kernel_size = 11  # Tamaño del filtro (ancho y alto)
stride = 4  # Stride
padding = 0  # Padding

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, stride=1, padding=1)  # 28x28x1 -> 28x28x10
        self.mp1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)  # 28x28x10 -> 14x14x10
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, stride=1, padding=1)  # 14x14x10 -> 14x14x20
        self.mp2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)  # 14x14x20 -> 7x7x20
        self.fc1 = nn.Linear(7 * 7 * 20, 100)  # 7x7x20 -> 100
        self.fc2 = nn.Linear(100, 10)  # 100 -> 10

    def forward(self, x):
        out_conv1 = self.conv1(x)
        x = F.relu(out_conv1)
        x = self.mp1(x)
        out_conv2 = self.conv2(x)
        x = F.relu(out_conv2)
        x = self.mp2(x)
        x = x.view(-1, 7 * 7 * 20)  # flatten
        # x = x.flatten()
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output, out_conv1, out_conv2

In [4]:
import torch

model = torch.hub.load("pytorch/vision:v0.10.0", "alexnet", pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to C:\Users\Sergi/.cache\torch\hub\v0.10.0.zip
c:\Users\Sergi\miniconda3\envs\ml-notes\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Sergi\miniconda3\envs\ml-notes\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\Sergi/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:09<00:00, 27.0MB/s] 


In [5]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## Entrenament

Aquesta part, per sort, no varia massa de la setmana anterior:

### Feina a fer

1. Modificar la sortida de la xarxa, ara retorna diversos valors, encara que aquí només us interessa un.


In [10]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    model.train()

    loss_v = 0

    for batch_idx, (data, target) in (t := tqdm(enumerate(train_loader), total=len(train_loader))):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data)
        loss = F.cross_entropy(output, target, reduction="sum")
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0 and verbose:
            t.set_description(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Average: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                    loss.item() / len(data),
                )
            )

        loss_v += loss.item()

    loss_v /= len(train_loader.dataset)
    print("\nTrain set: Average loss: {:.4f}\n".format(loss_v))

    return loss_v


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader, total=len(test_loader)):
            data, target = data.to(device), target.to(device)
            output, _, _ = model(data)
            test_loss += F.cross_entropy(output, target, reduction="sum")
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)
        )
    )

    return test_loss

A continuació definim els paràmetres d'entrenament i el bucle principal:

### Adam

Aquesta setmana introduirem un nou algorisme d'optimització anomenat `Adam`. Fins ara hem emprat el descens del gradient (`SGD`).

`Adam` és un algorisme d'optimització amplament emprat, tal com el descens del gradient, és iteratiu. A la literatura trobam arguments que indiquen que, tot i que Adam convergeix més ràpidament, SGD generalitza millor que Adam i, per tant, pot resultar en un rendiment final millor.

[Més info](https://medium.com/geekculture/a-2021-guide-to-improving-cnns-optimizers-adam-vs-sgd-495848ac6008)

### Feina a fer:

1. Mostrar el nombre de paràmetres de la xarxa i compara-ho amb el nombre de paràmetres amb la xarxa de la setmana passada). Fes la teva xarxa més petita fins que el resultat decaigui.

```
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
```

2. Dibuixar els gràfics de la funció de pèrdua amb les dues funcions d'optimització que coneixem: comparar `SGD` amb `ADAM`.


In [11]:
torch.manual_seed(33)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

epochs = 15
lr = 0.00001

model = Net().to(device)
print(model)

optimizer = optim.Adam(model.parameters(), lr=lr)

# Guardam el valor de peèrdua mig de cada iteració (època)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle d'entrenament
for epoch in range(0, epochs):
    train_l[epoch] = train(model, device, train_loader, optimizer, epoch)
    test_l[epoch] = test(model, device, test_loader)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (mp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=980, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)


  0%|          | 0/3750 [00:00<?, ?it/s]

Train Epoch: 0 [236800/240000 (99%)]	Loss: 17.726179, Average: 0.276972: 100%|██████████| 3750/3750 [03:48<00:00, 16.40it/s]



Train set: Average loss: 0.9170



100%|██████████| 400/400 [02:22<00:00,  2.81it/s]


...


Test set: Average loss: 0.0552, Accuracy: 39334/40000 (98%)



Train Epoch: 14 [236800/240000 (99%)]	Loss: 0.289820, Average: 0.004528: 100%|██████████| 3750/3750 [04:23<00:00, 14.23it/s] 



Train set: Average loss: 0.0551



100%|██████████| 400/400 [00:33<00:00, 11.77it/s]


Test set: Average loss: 0.0524, Accuracy: 39361/40000 (98%)



## Resultats

Aquí visualitzarem els resultats d'aprenentatge de la xarxa.

### Feina a fer:

1. Fer una predicció del primer _batch_ del conjunt de _test_.
2. Visualitzar una imatge del _batch_ i posar la predicció i el groun truth com a títol de la imatge.
3. Visualitzar el resultat de la mateixa imatge passada per tots els filtres de la primera convolució de la vostra xarxa.
4. **Extra**: Fer la matriu de confusió de les 10 classes per poder entendre el que no estau fent bé (la xarxa no està fent bé).

A tenir en compte:

#### Subplots

Per fer graelles d'imatges podeu empar la funció `subplots`. Més [informació](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html)

#### Device

Si heu emprat _GPU_ per accelerar el procés d'entrenament, els resultats que obtenim de la xarxa també seràn a la _GPU_. **Pytorch** proporciona la funció `cpu()` que retorna una còpia d'aquest objecte a la memòria de la CPU.

#### Detach

Per poder operar amb els resultats de la predicció emprarem la funció `detach` que retorna un nou Tensor "separat" del graf (xarxa) en curs.

Per tant per transformar el tensor que retorna la xarxa en un array de la lliberia _Numpy_ caldria fer el següent:

```
resultat_np = resultat.detach().numpy()
```

Si a més hem executat l'entrenament en _GPU_:

```
resultat_np = resultat.cpu().detach().numpy()
```


In [21]:
def generador(loader):
    for data, target in loader:
        yield data, target


data, target = next(generador(test_loader))

output, out_conv1, out_conv2 = model(data.to(device))

output.shape, out_conv1.shape, out_conv2.shape

(torch.Size([100, 10]),
 torch.Size([100, 10, 28, 28]),
 torch.Size([100, 20, 14, 14]))